# Formal Evaluation Results Processing

In [3]:
import os
import os.path as osp
import math
import numpy as np
import pandas as pd
pd.set_option('display.width', 200)

In [4]:
root = '/Users/jimmy/Projects/RL/My_RLlib_Algo/eval_results'

In [3]:
test_csv = pd.read_csv(osp.join(root, 'ccppo-concat_inter_0.csv'))

In [4]:
test_csv['seed'].unique()[0]

12000

`concat` 一个 algo scene 对应的所有 eval csv 文件并保留有用的列，然后返回

In [5]:
def one_algo_scene(algo, scene, root=root, verbose=False):
    algo_scene = f'{algo}_{scene}'
    count = 0
    succ = [] # 对所有一次性求std
    crash = []
    out = []
    all_file_names = os.listdir(root)
    csv_list = []
    for name in all_file_names:
        if algo_scene in name:
            count += 1
            csv_list.append(pd.read_csv(osp.join(root, name)))
    if verbose: print(f'Found {count} eval results(csv) of {algo_scene}.')

    data = pd.concat(csv_list)

    ret = pd.DataFrame(columns=['success_rate', 'num_failed', 'net_success'])

    # == Success Rate ==
    ret['success_rate'] = data['success_rate']*100

    # == Safety ==
    # ret["num_failed"] = (data["num_agents_crash"] + data["num_agents_out"] ) / data['num_agents_total']
    ret["num_failed"] = (data["crash_rate"] + data["out_rate"])*100
    # ret["num_failed"] = (data["crash_rate"] + data["out_rate"])*100
    # ret["num_failed"] = (data["num_agents_total"] - data["num_agents_success"]) / data['num_agents_total'] * 100

    ret['num_agents_crash'] = data['num_agents_crash']
    ret['num_agents_out'] = data['num_agents_out']
    ret['num_critical_fail'] =  data['num_agents_crash'] + data['num_agents_out']


    # == Efficiency ==
    data['num_agents_maxstep'] = data["num_agents_total"] - data["num_agents_success"] - data["num_agents_crash"] - data["num_agents_out"]
    ret["net_success"] = (
        # data["num_agents_success"] - data["num_agents_crash"] - data["num_agents_out"] - data['num_agents_maxstep']
        data["num_agents_success"] -ret["num_failed"]
        # data["num_agents_success"] - data["num_agents_crash"] - data["num_agents_out"]
    )

    return ret

## 1. Main Metrics Processing:

concat all csv:

In [41]:
csv_list = []
all_file_names = [ osp.join(root, name) for name in os.listdir(root) if 'csv' in  name ]
for name in all_file_names:
    csv_list.append(pd.read_csv(name))
data = pd.concat(csv_list)

In [42]:
data.columns

Index(['Unnamed: 0', 'velocity_step_mean_episode_min', 'velocity_step_mean_episode_mean', 'velocity_step_mean_episode_max', 'energy_step_mean_episode_min', 'energy_step_mean_episode_mean',
       'energy_step_mean_episode_max', 'num_neighbours_mean_episode_mean', 'num_neighbours_mean_episode_max', 'num_agents_total', 'num_agents_total_per_300_steps', 'success_rate',
       'num_agents_success', 'num_agents_success_per_300_steps', 'num_agents_failed_per_300_steps', 'episode_reward_mean', 'episode_reward_min', 'episode_reward_max', 'episode_cost_mean',
       'episode_cost_min', 'episode_cost_max', 'episode_cost_sum', 'crash_rate', 'num_agents_crash', 'out_rate', 'num_agents_out', 'episode_length_mean', 'success_episode_length_mean',
       'svo_estimate_deg_mean', 'svo_estimate_deg_min', 'svo_estimate_deg_max', 'svo_reward', 'path', 'ckpt_idx', 'algo', 'env', 'seed', 'trial', 'trial_path', 'should_wrap_copo_env',
       'should_wrap_cc_env', 'episode'],
      dtype='object')

### DataFrame Preview:

In [8]:
pth = 'eval_results/main/scpo/scpo_inter_0.csv'
pd.read_csv(pth)

,Unnamed: 0,velocity_step_mean_episode_min,velocity_step_mean_episode_mean,velocity_step_mean_episode_max,energy_step_mean_episode_min,energy_step_mean_episode_mean,energy_step_mean_episode_max,num_neighbours_mean_episode_mean,num_neighbours_mean_episode_max,num_agents_total,...,path,ckpt_idx,algo,env,seed,trial,trial_path,should_wrap_copo_env,should_wrap_cc_env,episode
0,0,0.038236,4.562022,12.148758,0.0,0.019695,0.061327,10.259000,14.200000,120,...,/Users/jimmy/Projects/RL/My_RLlib_Algo/eval_ch...,1620,scpo,Inter,8000,SCPOTrainer_Intersection_b4a01_00002_2_num_age...,/Users/jimmy/Projects/RL/My_RLlib_Algo/eval_ch...,False,False,1
1,1,0.045635,4.314084,12.655176,0.0,0.018410,0.064536,10.755203,15.666667,112,...,/Users/jimmy/Projects/RL/My_RLlib_Algo/eval_ch...,1620,scpo,Inter,8000,SCPOTrainer_Intersection_b4a01_00002_2_num_age...,/Users/jimmy/Projects/RL/My_RLlib_Algo/eval_ch...,False,False,2
2,2,0.043667,4.382802,13.485906,0.0,0.018882,0.070800,10.465078,14.733333,113,...,/Users/jimmy/Projects/RL/My_RLlib_Algo/eval_ch...,1620,scpo,Inter,8000,SCPOTrainer_Intersection_b4a01_00002_2_num_age...,/Users/jimmy/Projects/RL/My_RLlib_Algo/eval_ch...,False,False,3
3,3,0.044683,4.803002,12.470843,0.0,0.020763,0.062434,10.049279,15.600000,127,...,/Users/jimmy/Projects/RL/My_RLlib_Algo/eval_ch...,1620,scpo,Inter,8000,SCPOTrainer_Intersection_b4a01_00002_2_num_age...,/Users/jimmy/Projects/RL/My_RLlib_Algo/eval_ch...,False,False,4
4,4,0.037922,4.273456,13.593557,0.0,0.018257,0.071871,10.526657,14.545455,115,...,/Users/jimmy/Projects/RL/My_RLlib_Algo/eval_ch...,1620,scpo,Inter,8000,SCPOTrainer_Intersection_b4a01_00002_2_num_age...,/Users/jimmy/Projects/RL/My_RLlib_Algo/eval_ch...,False,False,5
5,5,0.042325,4.722755,11.572851,0.0,0.020307,0.056740,10.406641,14.133333,123,...,/Users/jimmy/Projects/RL/My_RLlib_Algo/eval_ch...,1620,scpo,Inter,8000,SCPOTrainer_Intersection_b4a01_00002_2_num_age...,/Users/jimmy/Projects/RL/My_RLlib_Algo/eval_ch...,False,False,6
6,6,0.042608,4.525044,14.042589,0.0,0.019423,0.075321,10.209147,14.600000,120,...,/Users/jimmy/Projects/RL/My_RLlib_Algo/eval_ch...,1620,scpo,Inter,8000,SCPOTrainer_Intersection_b4a01_00002_2_num_age...,/Users/jimmy/Projects/RL/My_RLlib_Algo/eval_ch...,False,False,7
7,7,0.046832,4.742610,13.081676,0.0,0.020404,0.067947,10.249140,14.333333,125,...,/Users/jimmy/Projects/RL/My_RLlib_Algo/eval_ch...,1620,scpo,Inter,8000,SCPOTrainer_Intersection_b4a01_00002_2_num_age...,/Users/jimmy/Projects/RL/My_RLlib_Algo/eval_ch...,False,False,8
8,8,0.039891,4.473956,13.762468,0.0,0.019259,0.072221,10.438623,14.600000,116,...,/Users/jimmy/Projects/RL/My_RLlib_Algo/eval_ch...,1620,scpo,Inter,8000,SCPOTrainer_Intersection_b4a01_00002_2_num_age...,/Users/jimmy/Projects/RL/My_RLlib_Algo/eval_ch...,False,False,9
9,9,0.042892,4.451032,12.195420,0.0,0.019005,0.061068,10.241181,14.600000,125,...,/Users/jimmy/Projects/RL/My_RLlib_Algo/eval_ch...,1620,scpo,Inter,8000,SCPOTrainer_Intersection_b4a01_00002_2_num_age...,/Users/jimmy/Projects/RL/My_RLlib_Algo/eval_ch...,False,False,10


In [29]:
def read_csv_recursively(root_dir):
    all_dataframes = []  # 用于存储所有读取的 DataFrame
    all_algo_envs = set()

    # 遍历根目录
    for root, dirs, files in os.walk(root_dir):
        for file in files:
            # 检查文件扩展名是否为 '.csv'
            if file.endswith('.csv'):
                file_path = os.path.join(root, file)
                # 使用 pandas 的 read_csv 读取 CSV 文件
                dataframe = pd.read_csv(file_path)
                # check algo and env
                assert 'algo' in dataframe
                assert 'env' in dataframe
                all_algo_envs.add(f"{dataframe['algo'].unique()[0].upper()}-{dataframe['env'].unique()[0]}")
                all_dataframes.append(dataframe)

    # 将所有的 DataFrame 进行合并
    result_df = pd.concat(all_dataframes, ignore_index=True)
    all_dataframes.sort(key=lambda k: k.split('-')[0])
    all_dataframes = sorted(all_dataframes)
    print(f'Successfully read {len(all_algo_envs)} algo-sences! \n{all_algo_envs}')

    return result_df


In [28]:
df = read_csv_recursively('eval_results/main')

ValueError: The truth value of a DataFrame is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [ ]:
def concat_csv_recursively(pth):
    if osp.isfile(pth) and osp.split(pth)[-1].split('.')[-1] == 'csv':
        return pd.read_csv(pth)
    elif osp.isdir(pth):
        concat_csv_recursively()

    else:
        raise NotImplementedError

      

In [6]:


def preprocess_data(root):
    # root: path for eval results' csv files, containing dirs grouped in algos, e.g.,
    # root/
    #    |- ippo
    #    |- scpo

    # read and concat every csv file in root:
    csv_list = []
    all_file_names = [ osp.join(root, name) for name in os.listdir(root) if 'csv' in name ]
    for name in all_file_names:
        csv_list.append(pd.read_csv(name))
    data = pd.concat(csv_list)

    sub_files = [osp.join(root, name) for name in os.listdir(root)] # may be csv or sub-dirs
    for _pth in sub_files:
        osp.isdir(_pth)
    

    # Success Rate
    data['SuccessRate'] = data['success_rate']*100

    # Safety
    data['cra_out_rate'] = (data['crash_rate'] + data['out_rate'])*100

    # Efficiency
    data['efficiency'] = (data['num_agents_success'] - data['num_agents_crash'] - data['num_agents_out']) / data['episode_length_mean']

    # Maxstep Rate
    data['maxstep_rate'] = (1 - data['success_rate'] - data['crash_rate'] - data['out_rate'])*100

    # pre-process nei_r
    data['nei_r'] = data.apply(lambda row: np.sqrt(row['svo_reward']**2 - row['episode_reward_mean']**2), axis=1)
    data['social_r'] = data['nei_r'] + data['episode_reward_mean']
    data['r_per_step'] = data['episode_reward_mean'] / data['episode_length_mean']
    data['sr_per_step'] = data['social_r'] / data['episode_length_mean']

    # Mean Social Reward
    data['mean_social_r'] = data['sr_per_step']

    return data

In [44]:
data = preprocess_data(root)

# print desirable cols
metric_names = ['success_rate', 'num_agents_crash', 'crash_rate', 'num_agents_out', 'out_rate', 'episode_reward_mean', 'nei_r', 'social_r', 'r_per_step', 'sr_per_step', 'svo_estimate_deg_mean', 'svo_reward']

table_results = pd.DataFrame(columns=['env', 'algo']+metric_names)

for env_name, df in data.groupby('env'):
    for algo_name, _df in df.groupby('algo'):
        # table_results.loc[f'{algo_name}', 'env'] = env_name
        _d = {}
        for metric in metric_names:
            # table_results.loc[f'{algo_name}', metric] = _df[metric].mean()
           _d[metric] = round(_df[metric].mean(), 3) 
        _d.update({
            'env': env_name,
            'algo': algo_name,
        })
        table_results.loc[len(table_results)] = _d

        # print(f'{algo_name:14}| {_df[metric].mean():.2f}({_df[metric].std():.2f})')


table_results['crash_&_out'] = table_results['num_agents_crash'] + table_results['num_agents_out'] 
table_results['cra_out_rate'] = table_results['crash_rate'] + table_results['out_rate'] 

for env_name, df in table_results.groupby('env'):
    print(f'\n[{env_name}]:')
    print(df.iloc[:, 1:])



[Inter]:
           algo  success_rate  num_agents_crash  crash_rate  num_agents_out  out_rate  episode_reward_mean    nei_r  social_r  r_per_step  sr_per_step  svo_estimate_deg_mean  svo_reward  \
0  ccppo-concat         0.699            21.354       0.207           7.802     0.077              105.589   93.560   199.149       0.322        0.608                 41.945     141.085   
1      ccppo-mf         0.701            20.236       0.203           7.186     0.073              105.454   92.998   198.452       0.310        0.584                 41.716     140.614   
2          copo         0.763            19.684       0.177           6.276     0.056              110.648   96.524   207.171       0.352        0.659                 41.508     146.838   
3          ippo         0.733            16.771       0.174           6.540     0.067              107.951   97.609   205.560       0.308        0.585                 42.174     145.561   
4          scpo         0.823            12.5

### Check checkpoint one-by-one: 

In [175]:
def sort_eval_res(algo_scene='scpo_round', key_idx=-3):
    scpo_round_files = [ osp.join(root, name) for name in os.listdir(root) if algo_scene in name ]

    ckp_eval_res = {}
    cnt = 0
    for name in scpo_round_files:
        # print()
        df = pd.read_csv(name)
        ckp_eval_res[name[:-4].split('_')[-1]] = df
        cnt += 1
    print(f'Finishing {cnt} ckp eval res csv file!')
    assert len(ckp_eval_res) == cnt

    metric_value_list = []
    for idx, df in ckp_eval_res.items():
        succ_rate = round(df['success_rate'].mean()*100, 2)
        critical_fail_rate = round((df['crash_rate'].mean()+df['out_rate'].mean())*100, 2)
        maxstep_rate = round(100 - succ_rate - critical_fail_rate, 2)
        svo_r = round(df['svo_reward'].mean(), 1)
        svo = round(df['svo_estimate_deg_mean'].mean(), 2)

        metric_value_list.append((
            f'idx-{idx:2}', 
            f"ckp-{df['seed'].unique()[0]}-{df['ckpt_idx'].unique()[0]}", 
            succ_rate,
            critical_fail_rate,
            maxstep_rate,
            svo_r,
            svo,
        ))

    sorted_list = sorted(metric_value_list, key=lambda x: x[key_idx], reverse=True)

    return sorted_list

In [184]:
def print_tuple(tlist):
    print(' '*28, f"{'succ':7}{'fail':7}{'maxs':6}{'svo_r':7}{' svo':5}")
    for t in tlist:
        s = ''
        for e in t:
            if isinstance(e, str):
                if 'idx' in e:
                    s += f'{e:7} '
                elif 'ckp' in e:
                    s += f'{e:16} | ' 
            else:
                s += f' {e:5} '
    
        print(s)

SCPO Round

In [197]:
print_tuple(sort_eval_res('scpo_round', key_idx=-4))

Finishing 27 ckp eval res csv file!
                             succ   fail   maxs  svo_r   svo 
idx-6   ckp-6000-1893    |  78.45  20.88   0.67  317.8  43.37 
idx-15  ckp-5000-1954    |  80.07  19.93    0.0  317.7  43.45 
idx-9   ckp-6000-1908    |  80.42  18.78    0.8  320.1  42.89 
idx-32  ckp-12000-1722   |  82.13  17.87    0.0  323.2   43.5 
idx-18  ckp-11000-1533   |  82.67  16.87   0.46  319.4  42.81 
idx-16  ckp-11000-1524   |  82.87  16.82   0.31  326.2  43.01 
idx-8   ckp-6000-1902    |  83.57  16.27   0.16  321.8  43.03 
idx-33  ckp-12000-1725   |  83.71  16.25   0.04  326.2  43.45 
idx-17  ckp-11000-1527   |  83.75  16.04   0.21  326.0  42.82 
idx-19  ckp-11000-1536   |  84.72  14.81   0.47  322.4   42.6 
idx-0   ckp-10000-1656   |   80.1  14.68   5.22  323.2  42.83 
idx-27  ckp-9000-1569    |  85.45   14.6  -0.05  330.0  43.13 
idx-7   ckp-6000-1899    |  84.81  14.55   0.64  331.6  42.73 
idx-22  ckp-8000-846     |  85.58  14.05   0.37  334.4  43.26 
idx-23  ckp-8000-849

In [187]:
print_tuple(sort_eval_res('copo_round', key_idx=-2))

Finishing 22 ckp eval res csv file!
                             succ   fail   maxs  svo_r   svo 
idx-8   ckp-12000-1000   |  80.11  15.92   3.97  345.0  45.38 
idx-6   ckp-12000-985    |  80.09   15.6   4.31  343.5  45.23 
idx-13  ckp-9000-1745    |  73.69  20.33   5.98  343.2  44.98 
idx-12  ckp-9000-1740    |  72.95  19.59   7.46  342.4  44.79 
idx-0   ckp-8000-1935    |  88.79  11.05   0.16  339.8  43.42 
idx-14  ckp-9000-1750    |   73.6  22.07   4.33  339.0  44.94 
idx-2   ckp-8000-1954    |  86.75  13.25    0.0  337.1   43.5 
idx-9   ckp-6000-1000    |  84.47   15.3   0.23  335.8  44.79 
idx-21  ckp-5000-1875    |  86.76  13.19   0.05  335.5  42.85 
idx-16  ckp-7000-1950    |  69.62  14.61  15.77  335.4  44.39 
idx-20  ckp-11000-1345   |  75.79  21.42   2.79  334.2  44.64 
idx-17  ckp-7000-1954    |  65.54  13.94  20.52  334.1  44.46 
idx-3   ckp-10000-1710   |  85.76  13.02   1.22  333.7  43.34 
idx-1   ckp-8000-1950    |  85.37  14.63   -0.0  332.4  43.79 
idx-15  ckp-7000-194

In [201]:
print_tuple(sort_eval_res('ccppo-concat_round', key_idx=-4))

Finishing 32 ckp eval res csv file!
                             succ   fail   maxs  svo_r   svo 
idx-21  ckp-11000-790    |  56.01  26.59   17.4  276.5  42.92 
idx-22  ckp-11000-795    |   58.6  25.02  16.38  283.9  42.79 
idx-12  ckp-12000-925    |  64.45  19.25   16.3  292.8  42.72 
idx-33  ckp-8000-1070    |   69.3  18.79  11.91  302.9  42.71 
idx-23  ckp-11000-800    |   64.1  18.05  17.85  290.4  42.27 
idx-20  ckp-11000-785    |  64.46  17.44   18.1  297.0  42.33 
idx-24  ckp-11000-805    |  66.81   16.8  16.39  296.6  42.27 
idx-34  ckp-8000-1080    |  72.31  16.55  11.14  305.9  42.24 
idx-26  ckp-6000-1330    |  70.25  15.74  14.01  289.6  42.26 
idx-10  ckp-12000-915    |  67.41  15.62  16.97  292.7  42.76 
idx-13  ckp-12000-930    |  63.58  15.45  20.97  289.1  42.52 
idx-11  ckp-12000-920    |  67.89  14.23  17.88  296.7  42.66 
idx-8   ckp-5000-1950    |   76.3   13.5   10.2  308.9  42.57 
idx-25  ckp-6000-1320    |  66.43  13.45  20.12  285.2  42.01 
idx-14  ckp-12000-19

In [199]:
print_tuple(sort_eval_res('ccppo-mf_round', key_idx=-4))

Finishing 36 ckp eval res csv file!
                             succ   fail   maxs  svo_r   svo 
idx-29  ckp-12000-1895   |  73.77  22.58   3.65  302.9   42.1 
idx-32  ckp-8000-1570    |  78.75   20.2   1.05  315.5  43.48 
idx-13  ckp-11000-1415   |  79.33  18.53   2.14  321.3  42.76 
idx-26  ckp-12000-1870   |  76.54  18.07   5.39  310.3  41.73 
idx-25  ckp-12000-1860   |  78.28  18.06   3.66  308.2  42.03 
idx-28  ckp-12000-1885   |  77.32   17.0   5.68  308.6  41.66 
idx-27  ckp-12000-1880   |   79.3  16.56   4.14  313.7  41.94 
idx-24  ckp-10000-1240   |   74.2  16.17   9.63  313.6  42.34 
idx-14  ckp-11000-1420   |  77.36   16.1   6.54  316.5  42.41 
idx-22  ckp-10000-1230   |   73.2  15.75  11.05  311.6  42.27 
idx-8   ckp-5000-1890    |  83.48  15.66   0.86  324.5  43.01 
idx-30  ckp-8000-1550    |  82.84  15.55   1.61  322.2  43.32 
idx-12  ckp-11000-1410   |  77.41  15.24   7.35  314.9  42.37 
idx-20  ckp-10000-1180   |  72.84  15.19  11.97  318.9  42.26 
idx-0   ckp-9000-183

### Formal Table:

In [46]:
data = preprocess_data(root)

algo_name_mapping = {
    'ippo': 'IPPO \cite{de2020independent}', 
    'ccppo-mf': 'MFPO \cite{yu2022surprising}',
    'ccppo-concat': 'MAPPO \cite{yang2018mean}', 
    'copo': 'CoPO \cite{peng2021learning}', 
    'scpo': 'SCPO (Ours)', 
}
metrics = {
    'Success Rate': 'SuccessRate', 
    'Efficiency': 'efficiency',
    'Safety': 'cra_out_rate',
    'Individual Return': 'episode_reward_mean', 
    'Social Return': 'social_r', 
    'Social Coordination': 'mean_social_r',
}
formal_table = pd.DataFrame(columns=['env', 'algo']+list(metrics.keys()))

LATEX = True
if LATEX:
    latex_pre = '$\scriptstyle \pm'
    latex_suf = '$'
else:
    latex_pre = '('
    latex_suf = ')'


for env_name, df in data.groupby('env'):
    for algo_name, _df in df.groupby('algo'):
        # table_results.loc[f'{algo_name}', 'env'] = env_name
        row = {
            'env': env_name,
            'algo': algo_name_mapping[algo_name],
            # 'Success Rate': _df[''].mean(),
        }
        if algo_name == 'scpo':
            latext_bf_pre = '\\textbf{'
            latext_bf_suf = '}'
        else:
            latext_bf_pre = ''
            latext_bf_suf = ''
        for met in metrics:
            _col =_df[metrics[met]]
            row.update({
                f'{met}': f"{latext_bf_pre}{_col.mean():.2f}{latext_bf_suf} {latex_pre} {_col.std():.2f}{latex_suf}"
            })
         
        formal_table.loc[len(formal_table)] = row
    
formal_table = formal_table.reindex([3, 0, 1, 2, 4, 8, 5, 6, 7, 9])
formal_table

,env,algo,Success Rate,Efficiency,Safety,Individual Return,Social Return,Social Coordination
3,Inter,IPPO \cite{de2020independent},73.33 $\scriptstyle \pm 14.01$,0.16 $\scriptstyle \pm 0.09$,24.07 $\scriptstyle \pm 10.40$,107.95 $\scriptstyle \pm 12.07$,205.56 $\scriptstyle \pm 21.19$,0.58 $\scriptstyle \pm 0.15$
0,Inter,MAPPO \cite{yang2018mean},69.93 $\scriptstyle \pm 12.55$,0.15 $\scriptstyle \pm 0.10$,28.39 $\scriptstyle \pm 9.93$,105.59 $\scriptstyle \pm 11.00$,199.15 $\scriptstyle \pm 20.47$,0.61 $\scriptstyle \pm 0.16$
1,Inter,MFPO \cite{yu2022surprising},70.09 $\scriptstyle \pm 16.64$,0.15 $\scriptstyle \pm 0.11$,27.58 $\scriptstyle \pm 14.00$,105.45 $\scriptstyle \pm 14.11$,198.45 $\scriptstyle \pm 25.52$,0.58 $\scriptstyle \pm 0.17$
2,Inter,CoPO \cite{peng2021learning},76.31 $\scriptstyle \pm 7.44$,0.19 $\scriptstyle \pm 0.06$,23.33 $\scriptstyle \pm 6.57$,110.65 $\scriptstyle \pm 6.45$,207.17 $\scriptstyle \pm 12.01$,0.66 $\scriptstyle \pm 0.10$
4,Inter,SCPO (Ours),\textbf{82.28} $\scriptstyle \pm 7.51$,\textbf{0.22} $\scriptstyle \pm 0.07$,\textbf{16.56} $\scriptstyle \pm 6.40$,\textbf{116.53} $\scriptstyle \pm 6.63$,\textbf{221.65} $\scriptstyle \pm 13.02$,\textbf{0.67} $\scriptstyle \pm 0.12$
8,Round,IPPO \cite{de2020independent},74.53 $\scriptstyle \pm 10.86$,0.09 $\scriptstyle \pm 0.04$,14.31 $\scriptstyle \pm 6.47$,225.32 $\scriptstyle \pm 14.10$,433.06 $\scriptstyle \pm 28.23$,0.74 $\scriptstyle \pm 0.11$
5,Round,MAPPO \cite{yang2018mean},70.62 $\scriptstyle \pm 12.02$,0.08 $\scriptstyle \pm 0.03$,14.05 $\scriptstyle \pm 6.34$,221.87 $\scriptstyle \pm 15.42$,424.84 $\scriptstyle \pm 30.15$,0.68 $\scriptstyle \pm 0.09$
6,Round,MFPO \cite{yu2022surprising},79.79 $\scriptstyle \pm 9.15$,0.12 $\scriptstyle \pm 0.04$,14.04 $\scriptstyle \pm 5.57$,233.96 $\scriptstyle \pm 10.99$,450.21 $\scriptstyle \pm 22.04$,0.83 $\scriptstyle \pm 0.12$
7,Round,CoPO \cite{peng2021learning},78.31 $\scriptstyle \pm 8.89$,0.11 $\scriptstyle \pm 0.04$,17.40 $\scriptstyle \pm 6.83$,238.57 $\scriptstyle \pm 11.27$,470.40 $\scriptstyle \pm 22.94$,0.87 $\scriptstyle \pm 0.11$
9,Round,SCPO (Ours),\textbf{84.47} $\scriptstyle \pm 6.25$,\textbf{0.14} $\scriptstyle \pm 0.04$,\textbf{13.84} $\scriptstyle \pm 5.78$,\textbf{240.18} $\scriptstyle \pm 9.64$,\textbf{463.69} $\scriptstyle \pm 18.30$,\textbf{0.94} $\scriptstyle \pm 0.11$


In [47]:
print(formal_table.iloc[:, 1:].to_latex(index=False, escape=False))

\begin{tabular}{lllllll}
\toprule
algo & Success Rate & Efficiency & Safety & Individual Return & Social Return & Social Coordination \\
\midrule
IPPO \cite{de2020independent} & 73.33 $\scriptstyle \pm 14.01$ & 0.16 $\scriptstyle \pm 0.09$ & 24.07 $\scriptstyle \pm 10.40$ & 107.95 $\scriptstyle \pm 12.07$ & 205.56 $\scriptstyle \pm 21.19$ & 0.58 $\scriptstyle \pm 0.15$ \\
MAPPO \cite{yang2018mean} & 69.93 $\scriptstyle \pm 12.55$ & 0.15 $\scriptstyle \pm 0.10$ & 28.39 $\scriptstyle \pm 9.93$ & 105.59 $\scriptstyle \pm 11.00$ & 199.15 $\scriptstyle \pm 20.47$ & 0.61 $\scriptstyle \pm 0.16$ \\
MFPO \cite{yu2022surprising} & 70.09 $\scriptstyle \pm 16.64$ & 0.15 $\scriptstyle \pm 0.11$ & 27.58 $\scriptstyle \pm 14.00$ & 105.45 $\scriptstyle \pm 14.11$ & 198.45 $\scriptstyle \pm 25.52$ & 0.58 $\scriptstyle \pm 0.17$ \\
CoPO \cite{peng2021learning} & 76.31 $\scriptstyle \pm 7.44$ & 0.19 $\scriptstyle \pm 0.06$ & 23.33 $\scriptstyle \pm 6.57$ & 110.65 $\scriptstyle \pm 6.45$ & 207.17 $\scrip

## Compare other metrics by algo & scene

In [7]:
def get_one_algo_scene(algo, scene, metric: str, root=root):
    algo_scene = f'{algo}_{scene}'

    csv_list = []
    count = 0
    all_file_names = os.listdir(root)
    for name in all_file_names:
        if algo_scene in name:
            count += 1
            csv_list.append(pd.read_csv(osp.join(root, name)))
    print(f'Found {count} eval results(csv) of {algo_scene}.')

    data = pd.concat(csv_list)

    ret = pd.DataFrame(columns=[metric])
    ret[metric] = data[metric]

    return ret

cancat all algo for one scene, then use `groupby()`:

In [12]:
def concat_all_algo(scene: str, root=root):
    csv_list = []
    count = 0
    all_file_names = os.listdir(root)
    for name in all_file_names:
        if scene in name:
            count += 1
            csv_list.append(pd.read_csv(osp.join(root, name)))
    print(f'Found {count} eval results(csv) of {scene}.')

    return pd.concat(csv_list)

In [130]:
data = concat_all_algo('round')

Found 164 eval results(csv) of round.


### Add some new columns to df

In [131]:
metrics = ['num_agents_total', 'episode_reward_mean', 'episode_length_mean', 'svo_estimate_deg_mean', 'svo_reward']


for metric in metrics:
    print(f'\n[{metric}]')
    print('-----------'*3)
    for algo_name, df in data.groupby('algo'):
        print(f'{algo_name:14}| {df[metric].mean():.2f}({df[metric].std():.2f})')


[num_agents_total]
---------------------------------
ccppo-concat  | 79.75(10.10)
ccppo-mf      | 90.67(12.19)
copo          | 93.48(10.19)
ippo          | 87.48(13.04)
scpo          | 103.07(11.80)

[episode_reward_mean]
---------------------------------
ccppo-concat  | 224.43(15.57)
ccppo-mf      | 233.99(11.44)
copo          | 239.44(11.37)
ippo          | 227.30(17.47)
scpo          | 237.51(10.71)

[episode_length_mean]
---------------------------------
ccppo-concat  | 638.14(59.94)
ccppo-mf      | 564.74(77.47)
copo          | 554.48(77.90)
ippo          | 587.68(78.01)
scpo          | 488.86(68.96)

[svo_estimate_deg_mean]
---------------------------------
ccppo-concat  | 42.34(0.74)
ccppo-mf      | 42.48(0.79)
copo          | 44.29(0.97)
ippo          | 42.35(0.79)
scpo          | 42.96(0.72)

[svo_reward]
---------------------------------
ccppo-concat  | 304.23(21.57)
ccppo-mf      | 318.47(16.04)
copo          | 334.25(16.61)
ippo          | 309.33(24.90)
scpo          | 324

In [132]:
metric = 'svo_estimate_deg_mean'
for algo_name, df in data.groupby('algo'):
    print(f'{algo_name:14}| {df[metric].mean():.2f}({df[metric].std():.2f}) | max: {df[metric].max():.2f} | min: {df[metric].min():.2f}')

ccppo-concat  | 42.34(0.74) | max: 44.76 | min: 40.42
ccppo-mf      | 42.48(0.79) | max: 45.72 | min: 40.42
copo          | 44.29(0.97) | max: 46.99 | min: 42.19
ippo          | 42.35(0.79) | max: 44.62 | min: 40.13
scpo          | 42.96(0.72) | max: 45.11 | min: 40.87


### Prepare data for new svo degree computation

In [133]:
columns = ['algo', 'episode_reward_mean', 'svo_reward', 'svo_estimate_deg_mean']
new_data = pd.DataFrame(columns=columns)
for col in columns:
    new_data[col] = data[col]

In [134]:
new_data = new_data.reset_index(drop=True)

In [135]:
new_data

,algo,episode_reward_mean,svo_reward,svo_estimate_deg_mean
0,ccppo-mf,234.449516,320.225008,43.166971
1,ccppo-mf,229.146679,312.868965,42.866129
2,ccppo-mf,246.006183,337.326604,42.643110
3,ccppo-mf,236.669085,322.253778,42.683916
4,ccppo-mf,252.252888,346.140469,42.325738
...,...,...,...,...
3275,copo,245.720786,335.227651,43.028670
3276,copo,254.324859,347.405680,42.542118
3277,copo,237.513363,325.527960,42.760549
3278,copo,247.145451,337.362508,42.915593


In [136]:
new_data['nei_r_new'] = new_data.apply(lambda row: np.sqrt(row['svo_reward']**2 - row['episode_reward_mean']**2), axis=1)
new_data['social_r'] = new_data['nei_r_new'] + new_data['episode_reward_mean']
new_data['svo_ratio_new'] = new_data.apply(lambda row: np.rad2deg(np.arctan2(row['nei_r_new'], row['episode_reward_mean'])), axis=1)
new_data['svo_diff'] = new_data['svo_estimate_deg_mean'] - new_data['svo_ratio_new']

In [137]:
metric = 'svo_ratio_new'
for algo_name, df in new_data.groupby('algo'):
    print(f'{algo_name:14}| {df[metric].mean():.2f}({df[metric].std():.2f}) | max: {df[metric].max():.2f} | min: {df[metric].min():.2f} | {df.svo_diff.mean():.2f}')

ccppo-concat  | 42.46(0.33) | max: 43.45 | min: 41.37 | -0.12
ccppo-mf      | 42.71(0.41) | max: 43.68 | min: 41.27 | -0.23
copo          | 44.22(0.88) | max: 45.58 | min: 42.29 | 0.07
ippo          | 42.69(0.43) | max: 43.88 | min: 41.35 | -0.34
scpo          | 43.00(0.35) | max: 44.02 | min: 41.99 | -0.04


In [138]:
metric = 'social_r'
for algo_name, df in new_data.groupby('algo'):
    print(f'{algo_name:14}| {df[metric].mean():.2f}({df[metric].std():.2f}) | max: {df[metric].max():.2f} | min: {df[metric].min():.2f}')

ccppo-concat  | 429.81(30.52) | max: 508.17 | min: 315.19
ccppo-mf      | 450.02(22.71) | max: 512.04 | min: 343.64
copo          | 472.60(23.51) | max: 536.92 | min: 399.70
ippo          | 437.10(35.26) | max: 527.25 | min: 324.29
scpo          | 458.99(20.12) | max: 522.90 | min: 393.45


In [120]:
new_data

NameError: name 'new_data' is not defined

In [ ]:
metric = 'social_r'
for algo_name, df in new_data.groupby('algo'):
    print(f'{algo_name:14}| {df[metric].mean():.2f}({df[metric].std():.2f}) | max: {df[metric].max():.2f} | min: {df[metric].min():.2f}')

ccppo-concat  | 429.81(30.52) | max: 508.17 | min: 315.19
ccppo-mf      | 450.02(22.71) | max: 512.04 | min: 343.64
copo          | 472.60(23.51) | max: 536.92 | min: 399.70
ippo          | 437.10(35.26) | max: 527.25 | min: 324.29
scpo          | 458.99(20.12) | max: 522.90 | min: 393.45


In [60]:
def norm(a, b):
    return math.sqrt(a**2 + b**2)

In [54]:
own_r_mean =  116.53
nei_r_mean = own_r_mean * np.cos(np.deg2rad(41.73))
print(nei_r_mean)

print('svo new', norm(own_r_mean, nei_r_mean))

svo_r = 156.95
nei_r_new = math.sqrt(svo_r**2 - own_r_mean**2)
print('nei_r_new', nei_r_new)
print('new svo', np.rad2deg(math.atan2(nei_r_new, own_r_mean)))

86.96514644999236
svo new 145.40349926005433
nei_r_new 105.13829749430032
new svo 42.058114315411856


In [55]:
own_r_mean = 105.59
nei_r_mean = own_r_mean * np.cos(np.deg2rad(41.75))
print(nei_r_mean)

print('svo new', norm(own_r_mean, nei_r_mean))

svo_r = 141.09
nei_r_new = math.sqrt(svo_r**2 - own_r_mean**2)
print('nei_r_new', nei_r_new)
print('new svo', np.rad2deg(math.atan2(nei_r_new, own_r_mean)))

78.77619823276486
svo new 131.7381399140274
nei_r_new 93.57959179222786
new svo 41.54910476349994


## 3 Ablation Studies

Concat csv files for all methods:

> We only conduct ablation studies in `Intersection`!

In [7]:
root_scpo_no_cpo = 'eval_results/ablation/scpo-no-cpo/inter'
_df = preprocess_data(root_scpo_no_cpo)
_df.algo.unique()

FileNotFoundError: [Errno 2] No such file or directory: 'eval_results/ablation/scpo-no-cpo/inter'

In [70]:
root_scpo_no_cpo = 'eval_results/ablation/scpo-no-cpo/inter'
_df = preprocess_data(root_scpo_no_cpo).apply(lambda x: x.algo = 'scpo-no-cpo')
_df.algo.unique()

SyntaxError: expression cannot contain assignment, perhaps you meant "=="? (1632867497.py, line 2)

In [68]:
_df.algo.unique()

array(['scpo-no-cpo'], dtype=object)

In [86]:
_l = []

# 处理算法名
# eval时 `algo name` 搞错了导致的 `scpo-no-cpo` 的名字变成了 `ABLATION`
_name_roots = {
    'scpo-no-sch': 'eval_results/ablation/scpo-no-sch/inter',
    'scpo-no-cpo': 'eval_results/ablation/scpo-no-cpo/inter',
}
for name, root in _name_roots.items():
    _df = preprocess_data(root)
    _df.algo = name
    _l.append(_df)

# read ablation-variants
roots_to_eval = dict(
    root_ippo_rs = 'eval_results/ablation/ippo-rs/inter',
    root_ippo_dc = 'eval_results/ablation/ippo-dc/inter',
)
for name, root in roots_to_eval.items():
    _l.append(preprocess_data(root))

# read scpo
data = preprocess_data('eval_results')
_l.append(data[(data['algo'] == 'scpo') & (data['env'] == 'Inter')].copy())
_l.append(data[(data['algo'] == 'ippo') & (data['env'] == 'Inter')].copy())

data = pd.concat(_l).reset_index()

In [87]:
print(data.algo.unique())
print(data.env.unique())

['scpo-no-sch' 'scpo-no-cpo' 'exp_IPPO-RS' 'exp_IPPO-DC' 'scpo' 'ippo']
['Inter']


In [88]:
print(data.columns)

Index(['index', 'Unnamed: 0', 'velocity_step_mean_episode_min', 'velocity_step_mean_episode_mean', 'velocity_step_mean_episode_max', 'energy_step_mean_episode_min', 'energy_step_mean_episode_mean',
       'energy_step_mean_episode_max', 'num_neighbours_mean_episode_mean', 'num_neighbours_mean_episode_max', 'num_agents_total', 'num_agents_total_per_300_steps', 'success_rate',
       'num_agents_success', 'num_agents_success_per_300_steps', 'num_agents_failed_per_300_steps', 'episode_reward_mean', 'episode_reward_min', 'episode_reward_max', 'episode_cost_mean',
       'episode_cost_min', 'episode_cost_max', 'episode_cost_sum', 'crash_rate', 'num_agents_crash', 'out_rate', 'num_agents_out', 'episode_length_mean', 'success_episode_length_mean',
       'svo_estimate_deg_mean', 'svo_estimate_deg_min', 'svo_estimate_deg_max', 'svo_reward', 'path', 'ckpt_idx', 'algo', 'env', 'seed', 'trial', 'trial_path', 'should_wrap_copo_env',
       'should_wrap_cc_env', 'episode', 'SuccessRate', 'cra_out_r

In [8]:
data = preprocess_data('eval_results')

In [9]:
algo_name_mapping = {
    'ippo': 'IPPO \cite{de2020independent}', 
    'exp_IPPO-RS': 'IPPO-RS',
    'exp_IPPO-DC': 'IPPO-DC',
    'scpo-no-cpo': 'SCPO w/o CPO',
    'scpo-no-sch': 'SCPO w/o SCH',
    'scpo': 'SCPO (Ours)', 
}
metrics = {
    'Success Rate': 'SuccessRate', 
    'Efficiency': 'efficiency',
    'Safety': 'cra_out_rate',
    # 'Individual Rewards': 'episode_reward_mean', 
    # 'Max-Step Rate': 'maxstep_rate',
    # 'Social Coordination': 'mean_social_r',
    # 'nei_r': 'nei_r',
    # 'social_r': 'social_r',
    # 'velocity_step_mean_episode_mean': 'velocity_step_mean_episode_mean',
}
formal_table = pd.DataFrame(columns=['env', 'algo']+list(metrics.keys()))

LATEX = True
# LATEX = False
if LATEX:
    latex_pre = '$\scriptstyle \pm'
    latex_suf = '$'
else:
    latex_pre = '('
    latex_suf = ')'


for env_name, df in data.groupby('env'):
    for algo_name, _df in df.groupby('algo'):
        # table_results.loc[f'{algo_name}', 'env'] = env_name
        row = {
            'env': env_name,
            'algo': algo_name_mapping[algo_name],
            # 'Success Rate': _df[''].mean(),
        }
        if algo_name == 'scpo':
            latext_bf_pre = '\\textbf{'
            latext_bf_suf = '}'
        else:
            latext_bf_pre = ''
            latext_bf_suf = ''
        for met in metrics:
            _col =_df[metrics[met]]
            row.update({
                f'{met}': f"{latext_bf_pre}{_col.mean():.2f}{latext_bf_suf} {latex_pre} {_col.std():.2f}{latex_suf}"
            })
         
        formal_table.loc[len(formal_table)] = row
    
formal_table = formal_table.reindex([2, 1, 0, 4, 5, 3])
formal_table

,env,algo,Success Rate,Efficiency,Safety
2,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN
0,Round,SCPO (Ours),\textbf{84.47} $\scriptstyle \pm 6.25$,\textbf{0.14} $\scriptstyle \pm 0.04$,\textbf{13.84} $\scriptstyle \pm 5.78$
4,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN


In [93]:
print(formal_table.iloc[:, 1:].to_latex(index=False, escape=False))

\begin{tabular}{llll}
\toprule
algo & Success Rate & Max-Step Rate & Social Coordination \\
\midrule
IPPO \cite{de2020independent} & 73.33 $\scriptstyle \pm 14.01$ & 2.60 $\scriptstyle \pm 6.89$ & 0.58 $\scriptstyle \pm 0.15$ \\
IPPO-RS & 70.03 $\scriptstyle \pm 19.53$ & 11.33 $\scriptstyle \pm 13.68$ & 0.41 $\scriptstyle \pm 0.13$ \\
IPPO-DC & 34.35 $\scriptstyle \pm 13.84$ & 48.00 $\scriptstyle \pm 14.39$ & 0.18 $\scriptstyle \pm 0.05$ \\
SCPO w/o CPO & 40.71 $\scriptstyle \pm 14.70$ & 24.87 $\scriptstyle \pm 20.61$ & 0.30 $\scriptstyle \pm 0.10$ \\
SCPO w/o SCH & 82.53 $\scriptstyle \pm 5.95$ & 0.01 $\scriptstyle \pm 0.93$ & 0.70 $\scriptstyle \pm 0.08$ \\
SCPO (Ours) & \textbf{82.28} $\scriptstyle \pm 7.51$ & \textbf{1.16} $\scriptstyle \pm 4.17$ & \textbf{0.67} $\scriptstyle \pm 0.12$ \\
\bottomrule
\end{tabular}



In [6]:
root_copo_new = 'eval_results/main'
copo_df = preprocess_data(root_copo_new)

In [11]:
copo_df.columns

Index(['Unnamed: 0', 'velocity_step_mean_episode_min', 'velocity_step_mean_episode_mean', 'velocity_step_mean_episode_max', 'energy_step_mean_episode_min', 'energy_step_mean_episode_mean',
       'energy_step_mean_episode_max', 'num_neighbours_mean_episode_mean', 'num_neighbours_mean_episode_max', 'num_agents_total', 'num_agents_total_per_300_steps', 'success_rate',
       'num_agents_success', 'num_agents_success_per_300_steps', 'num_agents_failed_per_300_steps', 'episode_reward_mean', 'episode_reward_min', 'episode_reward_max', 'episode_cost_mean',
       'episode_cost_min', 'episode_cost_max', 'episode_cost_sum', 'crash_rate', 'num_agents_crash', 'out_rate', 'num_agents_out', 'episode_length_mean', 'success_episode_length_mean',
       'svo_estimate_deg_mean', 'svo_estimate_deg_min', 'svo_estimate_deg_max', 'svo_reward', 'path', 'ckpt_idx', 'algo', 'env', 'seed', 'trial', 'trial_path', 'should_wrap_copo_env',
       'should_wrap_cc_env', 'episode', 'SuccessRate', 'cra_out_rate', 'ef

In [13]:
metric_names = ['success_rate', 'crash_rate', 'out_rate', 'episode_reward_mean', 'nei_r', 'social_r', 'r_per_step', 'sr_per_step', 'svo_reward']
for m in metric_names:
    print(m, copo_df[m].mean())

success_rate 0.7936694400748772
crash_rate 0.11250514184063172
out_rate 0.05182921837011751
episode_reward_mean 239.6286789789812
nei_r 233.38084123802255
social_r 473.0095202170038
r_per_step 0.4401449111165484
sr_per_step 0.8677886457822335
svo_reward 334.5329440057342


In [14]:
copo_df['social_r']

0    476.940252
1    473.103133
2    477.070990
3    499.089226
4    512.874528
        ...    
5    435.973702
6    443.401335
7    439.554141
8    445.021997
9    462.707414
Name: social_r, Length: 240, dtype: float64

## NEW EVAL:

In [15]:
root_new= 'eval_results/main'
new_df = preprocess_data(root_new)

In [17]:
new_df.columns

Index(['Unnamed: 0', 'velocity_step_mean_episode_min', 'velocity_step_mean_episode_mean', 'velocity_step_mean_episode_max', 'energy_step_mean_episode_min', 'energy_step_mean_episode_mean',
       'energy_step_mean_episode_max', 'num_neighbours_mean_episode_mean', 'num_neighbours_mean_episode_max', 'num_agents_total', 'num_agents_total_per_300_steps', 'success_rate',
       'num_agents_success', 'num_agents_success_per_300_steps', 'num_agents_failed_per_300_steps', 'episode_reward_mean', 'episode_reward_min', 'episode_reward_max', 'episode_cost_mean',
       'episode_cost_min', 'episode_cost_max', 'episode_cost_sum', 'crash_rate', 'num_agents_crash', 'out_rate', 'num_agents_out', 'episode_length_mean', 'success_episode_length_mean',
       'svo_estimate_deg_mean', 'svo_estimate_deg_min', 'svo_estimate_deg_max', 'svo_reward', 'path', 'ckpt_idx', 'algo', 'env', 'seed', 'trial', 'trial_path', 'should_wrap_copo_env',
       'should_wrap_cc_env', 'episode', 'SuccessRate', 'cra_out_rate', 'ef

In [18]:
new_df.algo.unique()

array(['copo', 'scpo'], dtype=object)

In [20]:
data = new_df.copy()
# print desirable cols
metric_names = ['success_rate', 'num_agents_crash', 'crash_rate', 'num_agents_out', 'out_rate', 'episode_reward_mean', 'nei_r', 'social_r', 'r_per_step', 'sr_per_step', 'svo_reward']

table_results = pd.DataFrame(columns=['env', 'algo']+metric_names)

for env_name, df in data.groupby('env'):
    for algo_name, _df in df.groupby('algo'):
        # table_results.loc[f'{algo_name}', 'env'] = env_name
        _d = {}
        for metric in metric_names:
            # table_results.loc[f'{algo_name}', metric] = _df[metric].mean()
           _d[metric] = round(_df[metric].mean(), 3) 
        _d.update({
            'env': env_name,
            'algo': algo_name,
        })
        table_results.loc[len(table_results)] = _d

        # print(f'{algo_name:14}| {_df[metric].mean():.2f}({_df[metric].std():.2f})')


table_results['crash_&_out'] = table_results['num_agents_crash'] + table_results['num_agents_out'] 
table_results['cra_out_rate'] = table_results['crash_rate'] + table_results['out_rate'] 

for env_name, df in table_results.groupby('env'):
    print(f'\n[{env_name}]:')
    print(df.iloc[:, 1:])


[Round]:
   algo  success_rate  num_agents_crash  crash_rate  num_agents_out  out_rate  episode_reward_mean    nei_r  social_r  r_per_step  sr_per_step  svo_reward  crash_&_out  cra_out_rate
0  copo         0.787            10.542       0.112           4.725     0.052              238.829  231.690   470.519       0.432        0.849     332.787       15.267         0.164
1  scpo         0.831            11.045       0.106           4.794     0.047              238.295  221.902   460.197       0.488        0.942     325.620       15.839         0.153
